# Authentication Endpoints Testing

This notebook provides comprehensive testing for all authentication endpoints in the FastAPI CRUD application.

## Endpoints Covered:
- POST `/auth/signup` - User registration
- POST `/auth/login/access-token` - User login
- POST `/auth/test-token` - Token validation
- POST `/auth/password-recovery` - Request password reset
- POST `/auth/reset-password` - Reset password with token
- POST `/auth/logout` - User logout
- POST `/auth/refresh-token` - Refresh access token

In [ ]:
# Import required libraries
import requests
import json
from datetime import datetime
import time
from pprint import pprint

# Configuration
BASE_URL = "http://localhost:8000"
API_V1_STR = "/api/v1"
AUTH_URL = f"{BASE_URL}{API_V1_STR}/auth"

# Test data
test_user = {
    "email": f"test_auth_{int(time.time())}@example.com",
    "password": "TestPassword123!",
    "first_name": "Test",
    "last_name": "User",
}

# Global variables for storing tokens
access_token = None
refresh_token = None

print(f"Testing authentication endpoints at: {AUTH_URL}")
print(f"Test user email: {test_user['email']}")

: 

## 1. User Registration (Signup)

Test user registration functionality.

In [ ]:
# Test user signup
def test_signup():
    url = f"{AUTH_URL}/signup"

    response = requests.post(url, json=test_user)

    print(f"POST {url}")
    print(f"Status Code: {response.status_code}")
    print(f"Response Headers: {dict(response.headers)}")

    if response.status_code == 201:
        print("✅ Signup successful!")
        data = response.json()
        pprint(data)
        return data
    else:
        print("❌ Signup failed!")
        print(f"Error: {response.text}")
        return None


signup_result = test_signup()

## 2. User Login

Test user authentication and token generation.

In [ ]:
# Test user login
def test_login():
    global access_token, refresh_token

    url = f"{AUTH_URL}/login/access-token"

    # Form data for OAuth2 password flow
    login_data = {"username": test_user["email"], "password": test_user["password"]}

    response = requests.post(url, data=login_data)

    print(f"POST {url}")
    print(f"Status Code: {response.status_code}")

    if response.status_code == 200:
        print("✅ Login successful!")
        data = response.json()
        access_token = data.get("access_token")
        refresh_token = data.get("refresh_token")

        print(f"Access Token: {access_token[:50]}...")
        print(f"Refresh Token: {refresh_token[:50] if refresh_token else 'None'}...")
        print(f"Token Type: {data.get('token_type')}")

        return data
    else:
        print("❌ Login failed!")
        print(f"Error: {response.text}")
        return None


login_result = test_login()

## 3. Token Validation

Test token validation endpoint.

In [ ]:
# Test token validation
def test_token_validation():
    if not access_token:
        print("❌ No access token available. Please run login first.")
        return None

    url = f"{AUTH_URL}/test-token"
    headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.post(url, headers=headers)

    print(f"POST {url}")
    print(f"Status Code: {response.status_code}")

    if response.status_code == 200:
        print("✅ Token is valid!")
        data = response.json()
        pprint(data)
        return data
    else:
        print("❌ Token validation failed!")
        print(f"Error: {response.text}")
        return None


token_validation_result = test_token_validation()

## 4. Password Recovery Request

Test password recovery request functionality.

In [ ]:
# Test password recovery request
def test_password_recovery():
    url = f"{AUTH_URL}/password-recovery"

    data = {"email": test_user["email"]}

    response = requests.post(url, json=data)

    print(f"POST {url}")
    print(f"Status Code: {response.status_code}")

    if response.status_code == 200:
        print("✅ Password recovery request sent!")
        data = response.json()
        pprint(data)
        return data
    else:
        print("❌ Password recovery request failed!")
        print(f"Error: {response.text}")
        return None


password_recovery_result = test_password_recovery()

## 5. Token Refresh

Test token refresh functionality.

In [ ]:
# Test token refresh
def test_token_refresh():
    if not refresh_token:
        print("❌ No refresh token available. Please run login first.")
        return None

    url = f"{AUTH_URL}/refresh-token"

    data = {"refresh_token": refresh_token}

    response = requests.post(url, json=data)

    print(f"POST {url}")
    print(f"Status Code: {response.status_code}")

    if response.status_code == 200:
        print("✅ Token refresh successful!")
        data = response.json()
        # Update tokens
        global access_token
        access_token = data.get("access_token")
        print(f"New Access Token: {access_token[:50]}...")
        return data
    else:
        print("❌ Token refresh failed!")
        print(f"Error: {response.text}")
        return None


token_refresh_result = test_token_refresh()

## 6. User Logout

Test user logout functionality.

In [ ]:
# Test user logout
def test_logout():
    if not access_token:
        print("❌ No access token available. Please run login first.")
        return None

    url = f"{AUTH_URL}/logout"
    headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.post(url, headers=headers)

    print(f"POST {url}")
    print(f"Status Code: {response.status_code}")

    if response.status_code == 200:
        print("✅ Logout successful!")
        data = response.json()
        pprint(data)

        # Clear tokens
        global access_token, refresh_token
        access_token = None
        refresh_token = None

        return data
    else:
        print("❌ Logout failed!")
        print(f"Error: {response.text}")
        return None


logout_result = test_logout()

## 7. Error Cases Testing

Test various error scenarios.

In [ ]:
# Test error cases
def test_error_cases():
    print("Testing error cases...\n")

    # 1. Login with invalid credentials
    print("1. Testing login with invalid credentials:")
    invalid_login = {"username": "invalid@example.com", "password": "wrongpassword"}
    response = requests.post(f"{AUTH_URL}/login/access-token", data=invalid_login)
    print(f"Status: {response.status_code}, Response: {response.text}\n")

    # 2. Access protected endpoint without token
    print("2. Testing protected endpoint without token:")
    response = requests.post(f"{AUTH_URL}/test-token")
    print(f"Status: {response.status_code}, Response: {response.text}\n")

    # 3. Access protected endpoint with invalid token
    print("3. Testing protected endpoint with invalid token:")
    headers = {"Authorization": "Bearer invalid_token"}
    response = requests.post(f"{AUTH_URL}/test-token", headers=headers)
    print(f"Status: {response.status_code}, Response: {response.text}\n")

    # 4. Duplicate signup
    print("4. Testing duplicate signup:")
    response = requests.post(f"{AUTH_URL}/signup", json=test_user)
    print(f"Status: {response.status_code}, Response: {response.text}\n")


test_error_cases()

## 8. Complete Test Summary

Run all tests and provide a summary.

In [ ]:
# Complete test suite
def run_complete_auth_test_suite():
    print("🧪 Running Complete Authentication Test Suite")
    print("=" * 50)

    results = {"timestamp": datetime.now().isoformat(), "tests": {}}

    # Create new test user for complete suite
    suite_user = {
        "email": f"suite_test_{int(time.time())}@example.com",
        "password": "SuiteTestPassword123!",
        "first_name": "Suite",
        "last_name": "Test",
    }

    # 1. Signup
    print("\n1. Testing Signup...")
    signup_response = requests.post(f"{AUTH_URL}/signup", json=suite_user)
    results["tests"]["signup"] = {
        "status_code": signup_response.status_code,
        "success": signup_response.status_code == 201,
    }
    print(
        f"   Result: {'✅ PASS' if results['tests']['signup']['success'] else '❌ FAIL'}"
    )

    # 2. Login
    print("\n2. Testing Login...")
    login_data = {"username": suite_user["email"], "password": suite_user["password"]}
    login_response = requests.post(f"{AUTH_URL}/login/access-token", data=login_data)
    results["tests"]["login"] = {
        "status_code": login_response.status_code,
        "success": login_response.status_code == 200,
    }
    print(
        f"   Result: {'✅ PASS' if results['tests']['login']['success'] else '❌ FAIL'}"
    )

    # Get token for subsequent tests
    if results["tests"]["login"]["success"]:
        token_data = login_response.json()
        suite_token = token_data.get("access_token")
        headers = {"Authorization": f"Bearer {suite_token}"}

        # 3. Token validation
        print("\n3. Testing Token Validation...")
        token_response = requests.post(f"{AUTH_URL}/test-token", headers=headers)
        results["tests"]["token_validation"] = {
            "status_code": token_response.status_code,
            "success": token_response.status_code == 200,
        }
        print(
            f"   Result: {'✅ PASS' if results['tests']['token_validation']['success'] else '❌ FAIL'}"
        )

        # 4. Logout
        print("\n4. Testing Logout...")
        logout_response = requests.post(f"{AUTH_URL}/logout", headers=headers)
        results["tests"]["logout"] = {
            "status_code": logout_response.status_code,
            "success": logout_response.status_code == 200,
        }
        print(
            f"   Result: {'✅ PASS' if results['tests']['logout']['success'] else '❌ FAIL'}"
        )

    # Summary
    print("\n" + "=" * 50)
    print("📊 TEST SUMMARY")
    print("=" * 50)

    total_tests = len(results["tests"])
    passed_tests = sum(1 for test in results["tests"].values() if test["success"])

    print(f"Total Tests: {total_tests}")
    print(f"Passed: {passed_tests}")
    print(f"Failed: {total_tests - passed_tests}")
    print(f"Success Rate: {(passed_tests/total_tests)*100:.1f}%")

    return results


test_results = run_complete_auth_test_suite()